In [15]:
import math
import multiprocessing
import os
import sys

import pyprind 
from bw2data.utils import recursive_str_to_unicode
from lxml import objectify, etree
from stats_arrays.distributions import *

In [143]:
fp_process = '/Users/michael.baer/Dropbox (On)/Michaels Data/ILCD-hackaton/ILCD/processes/d2fe899e-7fc0-49d3-a7cc-bbf8cad5439a_00.00.001.xml'
fp_flows = '/Users/michael.baer/Dropbox (On)/Michaels Data/ILCD-hackaton/ILCD/flows/0a51e24a-6201-47bb-b8f2-eb52bca60c83_03.00.004.xml'

In [212]:
tree = etree.parse(open(fp_flows, encoding="utf-8"))
namespaces_str = {'n':'http://lca.jrc.it/ILCD/Flow','common':'http://lca.jrc.it/ILCD/Common'}
xpath_str = '/flowDataSet/flowInformation/dataSetInformation/name/baseName'
xpath_str = '/flowDataSet/flowInformation/dataSetInformation/common:UUID'
xpath_str = '/flowDataSet/flowInformation/dataSetInformation/classificationInformation/common:elementaryFlowCategorization/common:category[@level=2]'
xpath_str = '/flowDataSet/modellingAndValidation/LCIMethod/typeOfDataSet'
xpath_str = '/flowDataSet/flowProperties/flowProperty[@dataSetInternalID=/flowDataSet/flowInformation/quantitativeReference/referenceToReferenceFlowProperty/text()]/meanValue/text()'
#xpath_str = '/flowDataSet/flowProperties/flowProperty[@dataSetInternalID=/flowDataSet/flowInformation/quantitativeReference/referenceToReferenceFlowProperty/text()]/referenceToFlowPropertyDataSet/@refObjectId'

xpath_str = "/".join([s if ':' in s or '(' in s or '@' in s[:1] else "n:"+s if s != '' else '' for s in xpath_str.split('/')])
r = tree.xpath(xpath_str,namespaces=namespaces_str)
assert len(r)==1
r[0]

'1.0'

In [206]:
xpath_str

'/n:flowDataSet/n:flowProperties/flowProperty[@dataSetInternalID=/n:flowDataSet/n:flowInformation/n:quantitativeReference/n:referenceToReferenceFlowProperty/text()]/n:referenceToFlowPropertyDataSet/@refObjectId'

In [ ]:
{
    'code':['processDataSet', 'processInformation', 'dataSetInformation', 'common:UUID'],
    'name': ['processDataSet', 'processInformation', 'dataSetInformation', 'name', 'baseName'],
    'location': ['processDataSet', 'processInformation', 'geography', 'locationOfOperationSupplyOrProduction', 'attribute|location'],
    'reference product': 
    'unit': 
    'exchanges': exchanges
    [
        {
            'name':XPath(),
            'location':XPath(),
            'reference product':XPath(),
            'unit':XPath(),
            'input':(DATABASE_NAME,EXCHANGE_ACTIVITY_HASH)
        }
        ]
}

Extract from bw2io / ecospold2 extractor

In [33]:
def getattr2(obj, attr):
    try:
        return getattr(obj, attr)
    except:
        return {}
        
def extract_activity(cls, dirpath, filename, db_name):
        root = objectify.parse(
            open(os.path.join(dirpath, filename), encoding="utf-8")
        ).getroot()
        if hasattr(root, "activityDataset"):
            stem = root.activityDataset
        else:
            stem = root.childActivityDataset

        comments = [
            cls.condense_multiline_comment(
                getattr2(stem.activityDescription.activity, "generalComment")
            ),
            (
                "Included activities start: ",
                getattr2(
                    stem.activityDescription.activity, "includedActivitiesStart"
                ).get("text"),
            ),
            (
                "Included activities end: ",
                getattr2(
                    stem.activityDescription.activity, "includedActivitiesEnd"
                ).get("text"),
            ),
            (
                "Geography: ",
                cls.condense_multiline_comment(
                    getattr2(stem.activityDescription.geography, "comment")
                ),
            ),
            (
                "Technology: ",
                cls.condense_multiline_comment(
                    getattr2(stem.activityDescription.technology, "comment")
                ),
            ),
            (
                "Time period: ",
                cls.condense_multiline_comment(
                    getattr2(stem.activityDescription.timePeriod, "comment")
                ),
            ),
        ]
        comment = "\n".join(
            [
                (" ".join(x) if isinstance(x, tuple) else x)
                for x in comments
                if (x[1] if isinstance(x, tuple) else x)
            ]
        )

        classifications = [
            (el.classificationSystem.text, el.classificationValue.text)
            for el in stem.activityDescription.iterchildren()
            if el.tag == u"{http://www.EcoInvent.org/EcoSpold02}classification"
        ]

        data = {
            "comment": comment,
            "classifications": classifications,
            "activity type": ACTIVITY_TYPES[
                int(stem.activityDescription.activity.get("specialActivityType") or 0)
            ],
            "activity": stem.activityDescription.activity.get("id"),
            "database": db_name,
            "exchanges": [
                cls.extract_exchange(exc)
                for exc in stem.flowData.iterchildren()
                if "parameter" not in exc.tag
            ],
            "filename": os.path.basename(filename),
            "location": stem.activityDescription.geography.shortname.text,
            "name": stem.activityDescription.activity.activityName.text,
            "synonyms": [
                s.text
                for s in getattr(stem.activityDescription.activity, "synonym", [])
            ],
            "parameters": dict(
                [
                    cls.extract_parameter(exc)
                    for exc in stem.flowData.iterchildren()
                    if "parameter" in exc.tag
                ]
            ),
            "authors": {
                "data entry": {
                    "name": stem.administrativeInformation.dataEntryBy.get(
                        "personName"
                    ),
                    "email": stem.administrativeInformation.dataEntryBy.get(
                        "personEmail"
                    ),
                },
                "data generator": {
                    "name": stem.administrativeInformation.dataGeneratorAndPublication.get(
                        "personName"
                    ),
                    "email": stem.administrativeInformation.dataGeneratorAndPublication.get(
                        "personEmail"
                    ),
                },
            },
            "type": "process",
        }
        return data
